# 安装部署

In [ ]:
!git clone https://www.modelscope.cn/datasets/BJQW14B/bs_challenge_financial_14b_dataset.git
!git clone https://github.com/jxysoft/ChallengeAI.git

In [ ]:
import os
get_ipython().system = os.system
# !nohup python Qwen/openai_api.py -c jxy/Tongyi-Finance-14B-Chat-Int4 &
# !nohup python Qwen/openai_api.py -c jxy/Tongyi-Finance-14B-Chat &
!nohup python Qwen/openai_api.py -c TongyiFinance/Tongyi-Finance-14B-Chat-Int4 &
# !nohup python Qwen/openai_api.py -c Qwen/Qwen-14B-Chat-Int4 --all_gpu &
# !nohup python Qwen/openai_api.py -c Qwen/Qwen-7B-Chat &

# install

In [ ]:
%pip install -U torch torchaudio torchdata torchtext torchvision torchinfo

In [ ]:
!pip install tiktoken
!pip install modelscope
!pip install transformers_stream_generator==0.0.4
!pip install optimum==1.12.0
!pip install auto-gptq
# %pip install transformers_stream_generator==0.0.4
# %pip install modelscope   #>=1.9.0
# %pip install auto-gptq  #==0.4.2
# %pip install optimum==1.12.0

In [ ]:
!git clone -b v1.0.8 https://github.com/Dao-AILab/flash-attention
!cd flash-attention && pip install .
# 下方安装可选，安装可能比较缓慢。
# Below are optional. Installing them might be slow.
# pip install csrc/layer_norm
# pip install csrc/rotary

In [ ]:
import torch
print(torch.__version__)

# model usage

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat-Int4')

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat-Int4')

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", trust_remote_code=True, bf16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cpu", trust_remote_code=True).eval()
# use auto mode, automatically select precision based on the device.
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cpu", trust_remote_code=True).eval()
# 模型加载指定device_map='cuda:0'，更改成device_map='auto'会使用所有可用显卡

# Specify hyperparameters for generation
model.generation_config = GenerationConfig.from_pretrained(model_dir, trust_remote_code=True)

response, history = model.chat(tokenizer, "请解释一下资产负债率", history=None)
print(response)
# 资产负债率是一个财务比率，用来衡量一个企业的负债水平。它是用一个企业负债总额除以其资产总额的百分比来表示的。它的计算公式是：资产负债率 = 负债总额 / 资产总额。它能够反映一个企业的财务状况，以及它是否具有足够的资产来抵偿其债务。

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B')

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda:0", trust_remote_code=True, bf16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cpu", trust_remote_code=True).eval()
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda:0", trust_remote_code=True).eval()
# 模型加载指定device_map='cuda:0'，更改成device_map='auto'会使用所有可用显卡

# Specify hyperparameters for generation
model.generation_config = GenerationConfig.from_pretrained(model_dir, trust_remote_code=True)

inputs = tokenizer('市盈率是最常用来评估股价水平是否合理的指标之一，是指', return_tensors='pt')
inputs = inputs.to(model.device)
pred = model.generate(**inputs)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))
# 市盈率是最常用来评估股价水平是否合理的指标之一，是指股票价格与每股盈利的比率。...

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat-Int4')
# model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat')


# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
print("begin load model")
# quantization_config=bnb_config,
# use bf16
model = AutoModelForCausalLM.from_pretrained(model_dir, trust_remote_code=True).eval()
# none int4
# model = AutoModelForCausalLM.from_pretrained(model_dir,  device_map="auto", offload_folder="offload", offload_state_dict = True, trust_remote_code=True, bf16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cpu", trust_remote_code=True).eval()
# use auto mode, automatically select precision based on the device.
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda:0", trust_remote_code=True).eval()
# 模型加载指定device_map='cuda:0'，更改成device_map='auto'会使用所有可用显卡
print("after load model ")
# Specify hyperparameters for generation
# model.generation_config = GenerationConfig.from_pretrained(model_dir, trust_remote_code=True)

# response, history = model.chat(tokenizer, "请解释一下资产负债率", history=None)
# print(response)
# 资产负债率是一个财务比率，用来衡量一个企业的负债水平。它是用一个企业负债总额除以其资产总额的百分比来表示的。它的计算公式是：资产负债率 = 负债总额 / 资产总额。它能够反映一个企业的财务状况，以及它是否具有足够的资产来抵偿其债务。

In [ ]:
from transformers import AutoTokenizer, AutoModel,AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("jxy/Tongyi-Finance-14B-Chat-Int4", 
                                          device_map="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("jxy/Tongyi-Finance-14B-Chat-Int4", torch_dtype="auto", 
    offload_folder="offload", device_map="cuda:0", 
    offload_state_dict = True, max_memory={0:"14GB",1:"14GB","cpu": "20GB"}, trust_remote_code=True)


In [ ]:
print("begin chat")
response, history = model.chat(tokenizer, "请解释一下资产负债率", history=None)
print(response)
response, history = model.chat(tokenizer, "立讯精密的资产负债率如何", history=history)
print(response)

In [ ]:
tokenizer.save_pretrained(push_to_hub=False, save_directory="/root/.cache/modelscope/model")

In [ ]:
model.save_pretrained(push_to_hub=False, save_directory="/root/.cache/modelscope/model", max_shard_size="2GB")

In [ ]:
from transformers import AutoTokenizer, AutoModel,AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True)

In [ ]:
model.save_pretrained(push_to_hub=False, save_directory="/root/.cache/modelscope/model", max_shard_size="2GB")